In [2]:
import requests

from bs4 import BeautifulSoup

from src.jobs import get_soup

In [4]:
url = 'https://www.emploi.ci/recherche-jobs-cote-ivoire'

In [12]:
soup = get_soup(url)

https://www.emploi.ci/recherche-jobs-cote-ivoire


In [14]:
def extract_job_information(soup):
    job_cards = soup.find_all('div', class_='card-job')

    jobs = []

    for card in job_cards:
        job = {}

        # Extracting job title
        title_elem = card.find('h3')
        job['title'] = title_elem.text.strip() if title_elem else None

        # Extracting company name
        company_elem = card.find('a', class_='card-job-company')
        job['company'] = company_elem.text.strip() if company_elem else None

        # Extracting job description
        desc_elem = card.find('div', class_='card-job-description')
        job['description'] = desc_elem.text.strip() if desc_elem else None

        # Extracting job details like education, experience, contract type, region, etc.
        details = card.find('ul').find_all('li') if card.find('ul') else []
        job['education'] = details[0].text.strip() if len(details) > 0 else None
        job['experience'] = details[1].text.strip() if len(details) > 1 else None
        job['contract_type'] = details[2].text.strip() if len(details) > 2 else None
        job['region'] = details[3].text.strip() if len(details) > 3 else None

        # Extracting job posting date
        time_elem = card.find('time')
        job['posting_date'] = time_elem.text.strip() if time_elem else None

        # Extracting job URL
        job['url'] = card.get('data-href')

        jobs.append(job)

    return jobs


In [15]:
extracted_jobs = extract_job_information(soup)

In [18]:
len(extracted_jobs)

25

In [41]:
def clean_job_details(jobs):
    cleaned_jobs = []

    for job in jobs:
        cleaned_job = {}

        for key, value in job.items():
            if key == 'url':
                cleaned_job[key] = value
            else:
                if ':' in value:
                    cleaned_value = value.split(':', 1)[-1].strip()
                else:
                    cleaned_value = value.strip()

                cleaned_job[key] = cleaned_value

        cleaned_jobs.append(cleaned_job)

    return cleaned_jobs

In [34]:
extracted_jobs[0]

{'title': 'Directeur Administratif et Financier Groupe - Abidjan',
 'company': 'AFRICAWORK',
 'description': 'RESPONSABILITÉS PRINCIPALES :   Le  Directeur Administratif et Financier  (DAF) est garant de la bonne gestion administrative et financière du Groupe. Il contrôle la bonne exécution de la stratégie et des procédures définies avec la direction générale et coordonne les services dont il a la charge.   Gestion administrative    Définir les principales orientations des systèmes de contrôle  \xa0Mettre en place les procédures de gestion et les indicateurs nécessaires au suivi des activités et au reporting à la direction  Veiller au respect des procédures et des délais  Responsable du recrutement, de la gestion et de la performance du personnel financier dans toutes les filiales  Contribuer au perfectionnement professionnel du... \n+plus',
 'education': 'Niveau d´études requis : Bac+5 et plus',
 'experience': "Niveau d'expérience : Expérience > 10 ans",
 'contract_type': 'Contrat pro

In [42]:
cleaned_jobs = clean_job_details([extracted_jobs[0]])
cleaned_jobs

[{'title': 'Directeur Administratif et Financier Groupe - Abidjan',
  'company': 'AFRICAWORK',
  'description': 'Le  Directeur Administratif et Financier  (DAF) est garant de la bonne gestion administrative et financière du Groupe. Il contrôle la bonne exécution de la stratégie et des procédures définies avec la direction générale et coordonne les services dont il a la charge.   Gestion administrative    Définir les principales orientations des systèmes de contrôle  \xa0Mettre en place les procédures de gestion et les indicateurs nécessaires au suivi des activités et au reporting à la direction  Veiller au respect des procédures et des délais  Responsable du recrutement, de la gestion et de la performance du personnel financier dans toutes les filiales  Contribuer au perfectionnement professionnel du... \n+plus',
  'education': 'Bac+5 et plus',
  'experience': 'Expérience > 10 ans',
  'contract_type': 'CDI',
  'region': 'Abidjan',
  'posting_date': '27.06.2024',
  'url': 'https://www.e

### Pagination

In [50]:
def extract_pagination_links(soup):
    pagination_div = soup.find('div', class_='pagination')

    if pagination_div:
        pagination_links = pagination_div.find_all('a')
        page_links = {'https://www.emploi.ci' + link.get('href') for link in pagination_links}
        return list(page_links)

    return []

In [51]:
pagination_links = extract_pagination_links(soup)
pagination_links

['https://www.emploi.ci/recherche-jobs-cote-ivoire?page=4',
 'https://www.emploi.ci/recherche-jobs-cote-ivoire?page=2',
 'https://www.emploi.ci/recherche-jobs-cote-ivoire?page=1']

### Educarrie

In [56]:
def extract_job_data(soup):
    job_data = []

    job_cards = soup.find_all('div', class_='rt-post post-md style-8')

    for card in job_cards:
        job = {}

        # Extracting the job URL
        job['url'] = card.find('a', href=True)['href']
        
        # Extracting the job category
        job['category'] = card.find('a', class_='racing').text.strip()

        # Extracting the job title
        job['title'] = card.find('h4', class_='post-title').text.strip()

        # Extracting job code
        job['code'] = card.find('span', style='color:#FF0000;font-size: 10px;').text.strip()

        # Extracting dates
        meta_items = card.find_all('li')
        for item in meta_items:
            if 'Date d\'édition' in item.text:
                job['date_edition'] = item.find('span', style='color:#FF0000;font-size: 10px;').text.strip()
            if 'Date limite' in item.text:
                job['date_limite'] = item.find('span', style='color:#FF0000;font-size: 10px;').text.strip()

        job_data.append(job)

    return job_data

In [52]:
url_educ = 'https://emploi.educarriere.ci/emploi/page/all'
soup_edu = get_soup(url_educ)

https://emploi.educarriere.ci/emploi/page/all


In [57]:
jobs_edu = extract_job_data(soup_edu)

In [58]:
jobs_edu[0]

{'url': 'https://emploi.educarriere.ci/offre-119127-avis-de-recrutement-d-agents-de-securite.html',
 'category': 'Emploi',
 'title': 'AGENTS DE SECURITE',
 'code': '114950',
 'date_edition': '28/06/2024',
 'date_limite': '28/07/2024'}

### mhd

In [64]:
def extract_job_data_from_cards(soup):
    job_data = []

    job_cards = soup.find_all('div', class_='cat_div')

    for card in job_cards:
        job = {}

        # Extracting the job URL
        job['url'] = card.find('a', href=True)['href']

        # Extracting the job title
        job['title'] = card.find('a').text.strip()

        # Extracting the seniority level
        seniority = card.find('p', class_='text-muted mb-1 mt-1 small')
        if seniority:
            job['seniority'] = seniority.text.strip()

        # Extracting the location
        location = card.find('p', class_='text-muted small')
        if location:
            job['location'] = location.text.strip()

        job_data.append(job)

    return job_data

In [60]:
url_mhd = 'https://mhd.hsd-melt.com/public/'
soup_mhd = get_soup(url_mhd)

https://mhd.hsd-melt.com/public/


In [62]:
jobs_mhd = extract_job_data_from_cards(soup_mhd)

In [65]:
jobs_mhd[0]

{'url': 'https://mhd.hsd-melt.com/public/public-search/1068',
 'title': "CHARGE D'AFFAIRES",
 'seniority': 'Niveau junior',
 'location': 'Libreville, Libreville,Gabon',
 'company_logo_url': 'https://mhd.hsd-melt.com/public/admin/images/logo/LOGO_HSD-Horizontale.png'}